# 計算ノードの削除

このnotebookでは、OCS OpenHPC-v2テンプレートで作成されたSlurmクラスタから計算ノードを削除する手順について説明します。

## 前提

* このnotebookで動作確認しているのは、mdx VM上に構築されたOpenHPC環境のみです。他のクラウドプロバイダ上に構築された環境に適用する場合には変更が必要かもしれません。
* 正常にジョブ実行ができる状態となっている状態を前提とします。構築途中であったり不具合のある状態の環境は対象としません。
* クラスタの管理を単純化するため、削除対象の計算ノードは、計算ノードのIPアドレスを符号なし32ビット整数として見た場合の最大のものから順に選択します。例えば、計算ノードが6ノードあり、IPアドレスがそれぞれ192.168.1.11-192.168.1.16である状態から2ノードを削除する場合、削除対象のノードは、192.168.1.16, 192.168.1.15のIPアドレスのノードとなります。

## 準備

### UnitGroup名の指定

構築環境の UnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために group_vars ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name = 

### VCCアクセストークンの入力

VCノードを起動するにはVC Controller(VCC)にアクセスして、操作を行う必要があります。VCCにアクセスするために必要となるアクセストークンをここで入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2021/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

> `unfreeze`するにはNotebookのツールバーにある`unfreeze below in section`ボタンなどを利用してください。

### group_varsの読み込み

次のセルを実行すると「010-パラメータの設定.ipynb」で指定したパラメータを読み込みます。読み込むパラメータの値は、UnitGroup名に指定した 値に対応するものになります。UnitGroup名の指定が誤っていると意図したパラメータが読み込めないので注意してください。

In [ ]:
%run scripts/group.py

gvars = load_group_vars(ugroup_name)

group_varsの内容を確認しておきます。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## 削除対象のノード抽出

削除するノード数を、`numnodes_remove`に設定します。この値は、group_varsのcompute_nodesより小さい値である必要があります。

In [ ]:
numnodes_remove = 1
if numnodes_remove >= gvars['compute_nodes']:
    raise RuntimeError('Error: numnodes_remove too large.')

削除する計算ノードを抽出します。[前提](#前提)の節に書いたように、IPアドレスの値の大きなものから順に選択します。

In [ ]:
import ipaddress

addrs_to_remove = sorted([ipaddress.ip_address(a)
                          for a in gvars['compute_etc_hosts'].keys()],
                         reverse=True)[0:numnodes_remove]
hosts_to_remove = [gvars['compute_etc_hosts'][str(a)]
                   for a in addrs_to_remove]
print(hosts_to_remove)

## ジョブスケジューリングの抑止

削除対象のノードにジョブがスケジュールされない状態に遷移させ、ジョブが割り当てられていない状態になるまで待ち合わせます。

削除対象の計算ノードをDRAIN状態に遷移させ、これ以上ジョブがスケジュールされないようにします。

In [ ]:
!ansible {ugroup_name}_master -b -a \
    "scontrol update NodeName={','.join(hosts_to_remove)} \
        State=DRAIN Reason='Node remboval'"

計算ノードの状態がDRAINEDになり、ジョブがスケジュールされない状態になるのを待ち合わせます。

In [ ]:
from time import sleep

for i in range(60):
    out = !ansible {ugroup_name}_master -b -a \
            "sinfo --noheader -o %T -N -n {','.join(hosts_to_remove)}" \
                | tail -n +2 | grep -v drained | wc -l
    if int(out[0]) == 0:
        break
    sleep(10)
else:
    raise RuntimeError('Error: timeout waiting for nodes drained.')

## Slurmクラスタの再構成

`slurm.conf`を削除対象のノードを除外した設定にし、Slurmクラスタの再構成を実施します。

`slurm.conf`の更新に先立って、計算ノードのホスト名のプレフィックス部分を推定します。以下のセルを実行することで、`c_hostname_prefix`にプレフィックス部分を設定します。設定内容が表示されます。

In [ ]:
cn_present = gvars['compute_etc_hosts']
cn_max = max(cn_present.values(), key=len)
for i in range(len(cn_max) - 2):
    c_hostname_prefix = cn_max[0:-(i+1)]
    for n in list(cn_present.values()):
        matched = n.startswith(c_hostname_prefix)
        if not matched:
            break
    if matched:
        break

c_hostname_prefix

表示されたプレフィックスが正しくない場合は、以下のセルのコメントを外して、c_hostname_prefixに正しいプレフィックスを設定してください。

In [ ]:
# c_hostname_prefix = ''

`/ec/slurm/slurm.conf`更新に先立ち、現状の`slurm.conf`のベースとなっているgroup_varsの設定を確認します。

In [ ]:
import json
print(json.dumps(gvars['slurm_conf'], indent=2))

`NodeName`に、削除ノードを外すように更新します。以下の処理では正しく更新できない場合には、別途セルを編集の上設定してください。

In [ ]:
from copy import deepcopy

compute_nodes_new = gvars['compute_nodes'] - numnodes_remove
gvars_new = deepcopy(gvars)
gvars_new['slurm_conf'].update(
    {'NodeName': f'{c_hostname_prefix}[1-{compute_nodes_new}]'}
)

print(json.dumps(gvars_new['slurm_conf'], indent=2))

各ノードの`slurm.conf`を更新します。更新に先立って必要なスクリプトを各ノードにコピーしています。

In [ ]:
# slurmfuncs.shを配布
!ansible {ugroup_name} -m copy -b -a \
    'src=scripts/slurmfuncs.sh dest=/etc/vcp/'

# 配布したスクリプトを使用してslurm.confを更新
%run scripts/utils.py
for addr in addrs_to_remove:
    del gvars_new['compute_etc_hosts'][str(addr)]
slurm_params = spec_env_slurm_conf(gvars_new)
!ansible {ugroup_name} -m file -b -a \
    'path=/var/lib/vcp/.20-slurm state=absent'
!ansible {ugroup_name} -m shell -b -a \
    ". /etc/vcp/slurmfuncs.sh; \
    SLURM_NODE_PARAMS={slurm_params} setup_slurm_conf && \
    MASTER_HOSTNAME={gvars_new['master_hostname']} setup_control_machine"

`slurmctld`を再起動します。

In [ ]:
!ansible {ugroup_name}_master -m systemd -b -a \
    'name=slurmctld state=restarted'

Slurmクラスタの再構成を指示します。

In [ ]:
!ansible {ugroup_name}_master -b -a \
    'scontrol reconfigure'

Slurmクラスタから削除対象のノードが削除されていることを確認します。

In [ ]:
!ansible {ugroup_name}_master -b -a \
    'sinfo'

## 削除対象のノードの終了処理

### VCノードの削除

計算ノードのUnitから、削除対象のVCノードを削除します。

計算ノードのUnitを取得します。

In [ ]:
ug = vcp.get_ugroup(gvars['ugroup_name'])
unit = ug.get_unit('compute')

現状のUnitの状態を確認します。

In [ ]:
unit.df_nodes()

対象のVCノードを削除します。

In [ ]:
unit.delete_nodes(
    ip_addresses=[str(a) for a in addrs_to_remove]
)

削除後のUnitの状態を表示し、削除対象のノードが削除されていることを確認します。

In [ ]:
unit.df_nodes()

### mdx VMの削除

mdxの操作のため、mdx REST API認証トークンを入力します。

In [ ]:
from getpass import getpass
mdx_token = getpass("mdx API token")

mdx REST APIエンドポイントにIPv6で接続しようとすると到達不可となる場合があるため、以下のセルを実行してIPv4での接続を強制します。

In [ ]:
def use_ipv4_only():
    import socket
    old_getaddrinfo = socket.getaddrinfo
    def new_getaddrinfo(*args, **kwargs):
        responses = old_getaddrinfo(*args, **kwargs)
        return [response
                for response in responses
                if response[0] == socket.AF_INET]
    socket.getaddrinfo = new_getaddrinfo

use_ipv4_only()

VCP SDK mdx用プラグインモジュールを読み込みます。

In [ ]:
from common import logsetting
from vcpsdk.plugins.mdx_ext import MdxResourceExt
mdx = MdxResourceExt(mdx_token)
mdx.set_current_project_by_name(gvars['mdx_project_name'])

mdx VMを削除するためにはまず停止状態にする必要があるため、VMを強制停止します。削除するVMは後で使うことはなく、また、停止にかかる時間が短い方が望ましいため、OSのシャットダウンを経由せず強制停止します。

In [ ]:
from vcpsdk.plugins.mdx_ext import SLEEP_COUNT
from vcpsdk.plugins.mdx_ext import SLEEP_TIME_SEC
from time import sleep

for vm in hosts_to_remove:
    mdx.power_off_vm(vm, wait_for=False)
for vm in hosts_to_remove:
    for i in range(SLEEP_COUNT):
        info = mdx.get_vm_info(vm)
        if info['status'] == 'PowerOFF':
            break
        sleep(SLEEP_TIME_SEC)
    else:
        raise RuntimeError(f'Error: VM {vm} not powered off.')

VMを削除します。

In [ ]:
for vm in hosts_to_remove:
    mdx.destroy_vm(vm, wait_for=False)
for vm in hosts_to_remove:
    for i in range(SLEEP_COUNT):
        info = mdx.get_vm_info(vm)
        if info is None:
            break
        sleep(SLEEP_TIME_SEC)
    else:
        raise RuntimeError(f'Error: VM {vm} not destroyed off.')

## notebook環境側のデータ更新

### ansibleの設定更新

削除したVCノードをansibleでの操作対象から外すため、ansibleの設定を更新します。

SSHのknown_hostsファイルから、削除したノードのエントリを削除します。

In [ ]:
for addr in addrs_to_remove:
    !ssh-keygen -R {addr}

ansibleの`inventory.yml`を更新します。

In [ ]:
import yaml

inventory = {
    'all': {
        'children': {
            ug.name: {
                'children': dict([
                    (
                        f'{ug.name}_{unit.name}',
                        {
                            'hosts': dict([
                                (ip, {})
                                for ip in unit.find_ip_addresses()
                            ])
                        }
                    )
                    for unit in ug.find_units()]),
                'vars': {
                    'ansible_user': 'vcp',
                    'ansible_ssh_private_key_file': gvars['ssh_private_key_path'],
                }
            }
        }
    }
}

with open('inventory.yml', 'w') as f:
    yaml.safe_dump(inventory, f, default_flow_style=False)
    
!cat inventory.yml

### group_varsの更新

ノードを削除した際に更新する必要がある、以下のパラメータを更新します。
* `slurm_conf`
* `compute_nodes`
* `compute_etc_hosts`

In [ ]:
%run scripts/group.py

update_group_vars(
    ugroup_name,
    slurm_conf=gvars_new['slurm_conf'],
    compute_nodes=compute_nodes_new,
    compute_etc_hosts=gvars_new['compute_etc_hosts']
)

group_varsの内容を確認します。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## `/etc/hosts`の更新

計算ノードの削除にあたっては必須の作業ではありませんが、Slurmクラスタのノードの`/etc/hosts`から、削除した計算ノードのエントリを削除します。

In [ ]:
for addr in addrs_to_remove:
    host = gvars['compute_etc_hosts'][str(addr)]
    !ansible {ugroup_name} -m lineinfile -b -a \
        "path=/etc/hosts line='{addr}\t{host}' \
        state=absent unsafe_writes=true"